<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/UserTimePyomoNeos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accessing computational time in PYOMO with NEOS

## Requirements

In [1]:
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc
import os
import random
import pyomo.environ as pe
os.environ['NEOS_EMAIL'] = 'salvapineda2@gmail.com'

     |████████████████████████████████| 9.5MB 5.6MB/s 
     |████████████████████████████████| 256kB 49.7MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 163kB 54.1MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 160975 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coi

## Create the optimization model in pyomo

In [2]:
# Dimensions
nvar = 10
ncon = 5
# Random vector and matrixes
c = [round(random.uniform(0,1),2) for i in range(nvar)]
A = [[round(random.uniform(-1,1),2) for i in range(nvar)] for j in range(ncon)]
b = [round(random.uniform(-10,10),2) for j in range(ncon)]
# Model
m = pe.ConcreteModel()
# Sets
m.i = pe.Set(initialize=range(nvar),ordered=True)
m.j = pe.Set(initialize=range(ncon),ordered=True)
# Variables
m.x = pe.Var(m.i,within=pe.NonNegativeIntegers)
# Objective function
def obj_rule(m):
  return sum(c[i]*m.x[i] for i in m.i)
m.obj = pe.Objective(rule=obj_rule)
# Constraints
def con_rule(m,j):
  return sum(A[j][i]*m.x[i] for i in m.i) <= b[j]
m.con = pe.Constraint(m.j,rule=con_rule)

## Solve the model with CBC

In [3]:
pe.SolverFactory('cbc', executable='/usr/bin/cbc').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 8.34
  Upper bound: 8.34
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 10
  Number of binary variables: 0
  Number of integer variables: 10
  Number of nonzeros: 10
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
 

## Solve the model with CPLEX and NEOS

In [4]:
pe.SolverManagerFactory('neos').solve(m,opt='cplex').write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 10
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: CPLEX 20.1.0.0\x3a optimal integer solution; objective 8.34; 6 MIP simplex iterations; 0 branch-and-bound nodes
  Termination condition: optimal
  Id: 2
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of 